In [82]:
import pandas as pd
import os
import sys
import numpy as np
import re

module_dir = "scripts" #change so it is the correct path
sys.path.append(module_dir)

from fasta_editing import fasta_to_df, fasta_writer

In [1]:
#this uses the master fasta file to parse out sequences by segment
#QC is done here but region and species data is done in the NCBI_QC function

def ncbi_prep(fasta_file, gene_segment_map, parsed_output):
    
    df = fasta_to_df(fasta_file)
    
    df['header'] = df['header'].str.replace(' ', '_')
    df['Strain'] = df['header'].str.split("|").str[0]
    df['Accession'] = df['header'].str.split("|").str[1]
    df['Subtype'] = df['header'].str.split("|").str[2]
    df['Date'] = df['header'].str.split("|").str[3]
    df['Host'] = df['header'].str.split("|").str[4]
    df['Country'] = df['header'].str.split("|").str[5]
    df['Segment'] = df['header'].str.split("|").str[6]

    #QC steps specific to NCBI
    df.Accession = df.Accession.str[:-2]
    df['Strain'] = df['Strain'].str.replace('>Influenza_A_virus_', '', regex=False)
    df['Strain'] = df['Strain'].str.extract(r'(\(.*?\))')
    df['Strain'] = df['Strain'].str.replace('^\(', '>', regex=True)
    df['Strain'] = df['Strain'].str.replace('\(\w+\)', '', regex=True)
    
    df = df[df["Date"] != "--"]
    df["Date"] = df["Date"].str.replace('/', '-')
    # print(df.Host.unique())
    
    df['header'] = df[['Strain', 'Accession', 'Subtype', 'Date', 'Host', 'Country']].apply('|'.join, axis=1)
    
    for segment, gene in gene_segment_map.items():
        segment_df = df[df['Segment'] == segment]
        fasta_writer(parsed_output, f"h3nx_{gene}.fasta", segment_df)

In [2]:
#this uses the master fasta file to parse out sequences by segment
#QC is done here but region and species data is done in the NCBI_QC function

def gisaid_prep(fasta_file, gene_segment_map, parsed_output):
    
    df = fasta_to_df(fasta_file)
    
    df['header'] = df['header'].str.replace(' ', '_')
    df['Strain'] = df['header'].str.split("|").str[0]
    df['Accession'] = df['header'].str.split("|").str[1]
    df['Date'] = df['header'].str.split("|").str[2]
    df['Segment'] = df['header'].str.split("|").str[3]
    df['Species'] = df['Strain'].str.split("/").str[1]
    df = df[~df["Species"].str.lower().str.contains("environment|mouse|ferret")]
    
    df = df[df["Date"] != "--"]
    df["Date"] = df["Date"].str.replace('/', '-')

    df['header'] = df[['Strain', 'Accession', 'Date', 'Species']].apply('|'.join, axis=1)
    
    for segment, gene in gene_segment_map.items():
        segment_df = df[df['Segment'] == segment]
        fasta_writer(parsed_output, f"h3nx_{gene}.fasta", segment_df)

In [85]:
#this function should be run to finish the QC needed for nextstrain (QC folder)
#it also will dedupe the sequences (dedupe folder) and can standardize dates if needed (consistent)

def ncbi_qc(list_of_genes, input_path, output_path, species_csv, 
            regions_csv, dedupe = True):
    
    genes = list_of_genes
    
    for gene in genes:
        
        df = fasta_to_df(f"{input_path}h3nx_{gene}.fasta")

        #this can be customized, change it based on how the headers are in your data
        df['header'] = df['header'].str.replace(' ', '_')
        df['Strain'] = df['header'].str.split("|").str[0]
        df['Accession'] = df['header'].str.split("|").str[1]
        df['Subtype'] = df['header'].str.split("|").str[2]
        df['Date'] = df['header'].str.split("|").str[3]
        df['Country'] = df['header'].str.split("|").str[5]
        df['Species'] = df['Strain'].str.split("/").str[1]
        
        #cleanup based on previous problems with data
        df.Country.replace('Viet_Nam', 'Vietnam' , inplace =True)    
        df = df[df["Country"] != ""]
        df = df[~df["Subtype"].str.contains("H3Nx|H3,mixed|mixed,H3|mixed,_H3|Mixed,H3|mixed.H3|H3N-")]
        df.Subtype.replace('H3N6,H3', 'H3N6', inplace =True)
        df = df[~df["Species"].str.lower().str.contains("animal|environment|ferret|mouse")]
        
        #adding Xs whereever dates are incomplete
        df['Date'] = df['Date'].str.replace(r'--', '-XX-XX', regex=True)
        df['Date'] = df['Date'].str.replace(r'-$', '-XX', regex=True)
        df['Date'] = df['Date'].str.replace(r"^(\d{4}-\d{2})$" , r"\1-XX", regex=True)
        df['Date'] = df['Date'].str.replace(r"^(\d{4})$" , r"\1-XX-XX", regex=True)
        
        #adding region data
        regions = pd.read_csv(regions_csv)
        df = df.merge(regions,left_on = df["Country"].str.lower(), right_on= regions["country"], how= "left")
        df.drop(['key_0'], axis=1, inplace =True)
        #NCBI for some reason has latin names for host while genbank had "canine" or "swine"
        #standardizing that here

        df = speciesClean(df, species_csv)
        
        df['header'] = df[['Strain', 'Accession', 'Subtype', 'Date', 'Host', 'country', 'region','Species', 'Broad', 'Order']].apply('|'.join, axis=1)
        
        fasta_writer(f"{output_path}QC/", f"h3nx_{gene}.fasta", df)
        
    if dedupe:
        fastaDeDupe(list_of_genes, f"{output_path}QC/", output_path)

In [86]:
#this function should be run to finish the QC needed for nextstrain (QC folder)
#it also will dedupe the sequences (dedupe folder) and can standardize dates if needed (consistent)
def gisaid_qc(list_of_genes, metadata_path, input_path, output_path, species_csv, 
              regions_csv,  dedupe = True):
    
    metadata = pd.read_csv(metadata_path)

    #replacing any spaces in the Isolate_Name column with underscores
    #adding the > character so that you can find matches in the .fasta file
    metadata['Isolate_Name'] = metadata['Isolate_Name'].str.replace(' ', '_')
    metadata['Isolate_Name'] = '>' + metadata['Isolate_Name'].astype(str)

    #extracting the country name as the second value in the location column
    #location is formatted continent/country/state/county)
    #drops any sequences where location or country data is not available
    metadata.dropna(subset=['Location'], inplace=True)
    metadata['Country'] = metadata['Location'].str.split('/').str[1].str.strip()
    metadata.dropna(subset=['Country'], inplace=True)

    genes = list_of_genes
    
    for gene in genes:
        
        df = fasta_to_df(f"{input_path}h3nx_{gene}.fasta")

        #make sure this matches your data
        df['Strain'] = df['header'].str.split("|").str[0]
        df['Accession'] = df['header'].str.split("|").str[1]
        df['Date'] = df['header'].str.split("|").str[2]
        df['Species'] = df['header'].str.split("|").str[3]
        
        #adding Xs whereever dates are incomplete
        df['Date'] = df['Date'].str.replace(r'--', '-XX-XX', regex=True)
        df['Date'] = df['Date'].str.replace(r'-$', '-XX', regex=True)
        df['Date'] = df['Date'].str.replace(r"^(\d{4}-\d{2})$" , r"\1-XX", regex=True)
        df['Date'] = df['Date'].str.replace(r"^(\d{4})$" , r"\1-XX-XX", regex=True)

        #merging metadata with df on Isolate_Name column, adding metadata columns youre interested in
        merged = pd.merge(df, metadata[['Isolate_Name', 'Subtype', 'Country']], left_on='Strain', right_on='Isolate_Name')
        
        #country + host QC and replacing spaces
        merged.Country.replace('United States', 'USA', inplace =True)
        merged.Country.replace('Korea, Republic of', 'South Korea' , inplace =True)
        merged.Country.replace('Russian Federation', 'Russia' , inplace =True)
        merged.Country.replace('Hong Kong (SAR)', 'Hong Kong', inplace =True)
        merged.Country.replace("Lao, People's Democratic Republic", "Laos", inplace =True)
        merged.Country = merged.Country.str.replace(' ', '_')
        merged['Subtype'] = merged['Subtype'].str.replace('A / ', '', regex=False)
        
        #adding region data
        regions = pd.read_csv(regions_csv)
        merged = merged.merge(regions,left_on = merged["Country"].str.lower(), right_on= regions["country"], how= "left")

        merged.drop(['key_0'], axis=1, inplace =True)
        
        merged = speciesClean(merged, species_csv)
        
        #the fields are in the same order as in the ncbi QC, just named differently
        merged['header'] = merged[['Strain', 'Accession', 'Subtype', 'Date', 'Host', 'Country', 'region', 'Species', 'Broad','Order']].apply('|'.join, axis=1)
        
        fasta_writer(f"{output_path}QC/", f"h3nx_{gene}.fasta", merged) 

    if dedupe:
        fastaDeDupe(list_of_genes, f"{output_path}QC/", output_path)

In [87]:
#the purpose of this function is to remove duplicates by keeping the longest sequence
#while also keeping the most complete date
#this is because different researchers sometimes upload the same strain but the date 
#will be incomplete in one vs the other

#by default, it also calls standardize_dates which makes sure all strains ACROSS fasta files
#have the most complete date associated with each strain

def fastaDeDupe(list_of_genes, input_path, output_path):
    
    genes = list_of_genes

    for gene in genes: #make sure your file names formatted as h3nx_[gene}.fasta
        
        df = fasta_to_df(f"{input_path}h3nx_{gene}.fasta")
        
        df['strain'] = df['header'].str.split("|").str[0].str.lower()
        df['date'] = df['header'].str.split("|").str[3]

        # to double check dupes were taken out
        # duplicated_strains = df[df.duplicated(subset="strain")][['strain','sequence']]
        # duplicated_strains.strain.unique()
        
        # group by sequence length and date completeness, it keeps the longest sequence and the 
        # most complete date
        temp_df = df.groupby(['strain']).agg({
            "sequence": lambda s: max(s, key=len),
            "date": lambda s: max(s.str.replace("XX", ""), key=len)
        })
        
        new_df = temp_df.merge(right=df, on=["strain", "sequence"], how="inner", suffixes=["", "_OLD"])
         
        new_df["header"]= new_df.apply(lambda x: x['header'].replace(str(x['date_OLD']), str(x['date'])), axis=1)
        
        new_df = new_df.loc[:,~new_df.columns.str.endswith('_OLD')]
        
        #if the date and sequence are the same between duplicates, it wont drop it above
        #this line will make sure ALL duplicates are finally dropped
        new_df.drop_duplicates(subset=['strain'], keep='first', inplace=True, ignore_index=True)
        
        fasta_writer(f"{output_path}/deduped/", f"h3nx_{gene}.fasta", new_df)
        
        standardize_dates(list_of_genes, f"{output_path}/deduped/", f"{output_path}/consistent/") 

In [88]:
def standardize_dates(list_of_genes,input_path, output_path):
    
    genes = list_of_genes
    
    df = pd.DataFrame()

    for gene in list_of_genes:
        gene_df = fasta_to_df(f"{input_path}h3nx_{gene}.fasta")
        gene_df["gene"] = gene
        df = pd.concat([df, gene_df], ignore_index=True)

    df['strain'] = df['header'].str.split("|").str[0].str.lower()
    df['date'] = df['header'].str.split("|").str[3].str.replace("XX", "")
    df['dateX'] = df['header'].str.split("|").str[3]

    df['new_date'] = df.groupby('strain')['date'].transform('max')

    df['new_date'] = df['new_date'].str.replace(r'--', '-XX-XX', regex=True)
    df['new_date'] = df['new_date'].str.replace(r'-$', '-XX', regex=True)

    df['header'] =  df.apply(lambda x: x['header'].replace(str(x['dateX']), str(x["new_date"])), axis=1)

    for gene in list_of_genes:
        gene_df = df[df['gene'] == gene]
        fasta_writer(output_path, f"h3nx_{gene}.fasta", gene_df)


In [89]:
def speciesClean(df, species_csv):
    
    species_df = pd.read_csv(species_csv)
    
    species_df['annotated_lower'] = species_df['annotated'].str.lower()
    species_df['correction_lower'] = species_df['correction'].str.lower()
    df['Species_lower'] = df['Species'].str.lower()
    
    merged_df = df.merge(species_df, 
                         left_on='Species_lower', 
                         right_on='annotated_lower', 
                         how='left')
    
    # this will raise an error if there are species in the passed df that are not found in species_df.annotated.
    missing_species = merged_df.loc[merged_df['host'].isnull(), 'Species'].unique()
    if len(missing_species) > 0:
        raise ValueError(f"You need to update species.csv to include: {missing_species}")
    
    merged_df['Species'] = merged_df['correction'].fillna(merged_df['Species']).str.lower()
    
    merged_df['Host'] = merged_df['host']
    merged_df['Broad'] = merged_df['broad']
    merged_df['Order'] = merged_df['order']
    
    result_df = merged_df.drop(['Species_lower', 'annotated_lower', 'correction_lower', 
                                'correction', 'host', 'broad', 'order'], axis=1)
    
    result_df.drop_duplicates(subset=['Strain'], keep='first', inplace=True, ignore_index=True)
    
    return result_df


In [90]:
#this function takes in your gisaid and you genbank data, assuming all QC has been done on both, and appends 
#the gisaid data to the genbank, then calls the deDupe function 
def merge(list_of_genes, gisaid_path, NCBI_path, merged_path, dedupe=True):
    
    genes = list_of_genes
    
    try:  
        os.mkdir(merged_path)

    except OSError as error:
        pass
    
    for gene in genes:
        with open(f"{NCBI_path}h3nx_{gene}.fasta" , 'r') as f2, open(f"{gisaid_path}h3nx_{gene}.fasta", 'r') as f1, open(f"{merged_path}h3nx_{gene}.fasta", 'w') as f3:
            f3.write(f2.read())
            f3.write(f1.read())

    if dedupe:
        fastaDeDupe(list_of_genes, merged_path, merged_path)

In [91]:
#this function takes in your gisaid and you genbank data, assuming all QC has been done on both, and appends 
#the gisaid data to the genbank, then calls the deDupe function 
def mergeWithMA(list_of_genes, current_sequence_path, current_sequences_path, merged_path, dedupe=True):
    
    genes = list_of_genes
    
    try:  
        os.mkdir(merged_path)

    except OSError as error:
        pass
    
    for gene in genes:
        with open(f"{current_sequence_path}h3nx_{gene}.fasta" , 'r') as f2, open(f"{current_sequences_path}h3nx_{gene}.fasta", 'r') as f1, open(f"{merged_path}h3nx_{gene}.fasta" , 'a+') as f3:
            f3.write(f2.read())
            f3.write(f1.read())

    if dedupe:
        fastaDeDupe(list_of_genes, merged_path, merged_path)

In [92]:
#first need to download the data from ncbi and gisaid separately 

def wrapper_func(list_of_genes, gene_segment_map, data_main_folder, species_csv, regions_csv,
                 ncbi = False, ncbi_data_path = None, ncbi_data_filename = None, 
                 gisaid = False, gisaid_data_path = None, gisaid_data_filename = None, 
                 gisaid_metadata_filename = None, master_merge = False, master_data_path = None
                ):


    if ncbi and (ncbi_data_path is None or ncbi_data_filename is None):
        raise ValueError("NCBI paths must be specified when 'ncbi' is True.")
    if gisaid and (gisaid_data_path is None or gisaid_data_filename is None or gisaid_metadata_filename is None):
        raise ValueError("GISAID paths must be specified when 'gisaid' is True.")
    if master_merge and master_data_path is None:
        raise ValueError("Master merge path must be specified when 'master_merge' is True.")


    if ncbi:
        
        ncbi_prep(f"{data_main_folder}{ncbi_data_path}{ncbi_data_filename}", 
                  gene_segment_map, 
                  f"{data_main_folder}{ncbi_data_path}parsed/"
                 )
        
        ncbi_qc(list_of_genes, 
                f"{data_main_folder}{ncbi_data_path}parsed/", 
                f"{data_main_folder}{ncbi_data_path}", 
                species_csv,
                regions_csv,
                dedupe = True,
               )

    if gisaid:
        
        gisaid_prep(f"{data_main_folder}{gisaid_data_path}{gisaid_data_filename}", 
                    gene_segment_map, 
                    f"{data_main_folder}{gisaid_data_path}parsed/"
                   ) 
        
        gisaid_qc(list_of_genes, 
                  f"{data_main_folder}{gisaid_data_path}{gisaid_metadata_filename}", 
                  f"{data_main_folder}{gisaid_data_path}parsed/", 
                  f"{data_main_folder}{gisaid_data_path}",
                  species_csv,
                  regions_csv,
                  dedupe = True
                 )
        
    if gisaid and ncbi:
        
        merge(list_of_genes,
              f"{data_main_folder}{gisaid_data_path}consistent/", 
              f"{data_main_folder}{ncbi_data_path}consistent/",
              f"{data_main_folder}merged/"
             )

    # need to make standardize more efficient becuase it doesnt work on big fasta files
    # since it iterates over every unique strain
    if master_merge:
        
        mergeWithMA(list_of_genes,
                    master_data_path, 
                    f"{data_main_folder}merged/consistent/",
                    "./master_merged/",
                    dedupe = True
                   )

In [93]:
list_of_genes = ["ha", "pb1", "pb2","pa","mp","np","na","ns"]

gene_segment_map = {
    "1" : "pb2",
    "2" : "pb1",
    "3" : "pa",
    "4" : "ha",
    "5" : "np",
    "6" : "na",
    "7" : "mp",
    "8" : "ns"
}

# before you run this, make sure you have downloaded sequences from either gisaid or ncbi
# the data pulls must have a segment number as the last field so it can be parsed into segment
# specific fasta files.
# qc_pipeline should be outside of a main folder (data_pulls for example) which then has an ncbi and/or 
# gisaid subfolder as well
# also save the .xls of gisaid metadata as a .csv for the gisaid QC steps


wrapper_func(list_of_genes, 
             gene_segment_map, 
             data_main_folder = "./data_pulls/", 
             species_csv = "./species.csv",
             regions_csv = "./regions.csv",
             ncbi = True, 
             ncbi_data_path = "ncbi/", 
             ncbi_data_filename = "all_ncbi.fasta", 
             gisaid = True, 
             gisaid_data_path = "gisaid/", 
             gisaid_data_filename = "all_gisaid.fasta", 
             gisaid_metadata_filename = "all_gisaid.csv", 
             master_merge = True,
             master_data_path = "./new_master/",
            )